In [ ]:
using PicoQuant

## Contraction functions

In this notebook we provide some details on the different contraction functions in QuantEx and how they are used. We then illustrate how these contraction functions may be compared. We begin by creating a function to generate toy tensor networks which we can contract. 

In [ ]:
# Function for creating an n qubit GHZ circuit
function create_basic_circuit(n::Int)
    circuit_network = TensorNetworkCircuit(n)
    
    for i = 1:n
        add_gate!(circuit_network, gate_tensor(:H), [i])
    end
    
    for i = 1:n-1
        add_gate!(circuit_network, gate_tensor(:CX), [i, i+1])
    end
    
    add_input!(circuit_network, "0"^n)
    add_output!(circuit_network, "0"^n)
    
    circuit_network
end

### Full Wavefunction Contraction

One of the more basic contraction functions is `full_wavefunction_contraction!`. To use this function, the tensor network circuit we want to contract must contain input nodes. This function operates by first contracting all input nodes together to produce a a single node representing the wavefunction of the input state. Then, all of the quantum gate nodes in the network are contracted with the wavefucntion node in the order in which they appear in the associated quantum circuit.

In [ ]:
circuit_network = create_basic_circuit(3)

full_wavefunction_contraction!(circuit_network)

### Netcon Contraction

The `netcon_contraction!` function uses the netcon implementation in TensorOperation.jl to find the optimal contraction plan for the given tensor network. The network is then contracted according to this contraction plan. The netcon method is based on an exhaustive search of all possible contraction plans, the number of which grows rapidly with the number of node in the tensor network. The largest network this method will work for contains 36 nodes.

In [ ]:
circuit_network = create_basic_circuit(3)

netcon_contraction!(circuit_network)

### BGreedy Contraction

The Boltzmann greedy function, `bgreedy_contraction!`, will return the best of `N` contraction plans, for the given tensor network, which are sampled using the Boltzmann greedy method. This method builds a contraction plan by always randomly selecting a pair of tensors to contract next using a probabilty distribution which favours cheap contractions. In particular, the probability that a pair of tensors $A$ and $B$ are selected for contraction is proportional to a Boltzmann type distribution where the cost of contraction plays the role of energy:
\begin{equation} P(A, B) \propto \exp(-\mbox{cost}_\alpha(A, B)/\tau), \end{equation}

where the contraction cost is given by the following formula ($C$ denotes the tensor resulting from contracting $A$ and $B$):
\begin{equation} \mbox{cost}_\alpha(A, B) = \mbox{size}(C) - \alpha (\mbox{size}(A) + \mbox{size}(B)) \end{equation}

In [ ]:
circuit_network = create_basic_circuit(3)

α=1; τ=1; N=10

bgreedy_contraction!(circuit_network, α, τ, N)

### QuickBB Contraction

It was shown by Markov and Shi (https://doi.org/10.1137/050644756) that finding the optimal contraction plan for a tensor network is closely related to computing the tree width of the networks associated line graph. Thus, algorithms for finding tree decompositions of graphs, with a minimal tree width, can be adapted to find an optimal contraction plan for a tensor network. The `QuickBB_contraction!` does this by first creating a MetaGraph for the associated line graph of the given tensor network. Then the 'Quick Branch and Bound' method for finding the tree width of a graph, is used to find an edge elimination ordering for the network. This method was introduced by Gogate and Dechter in https://arxiv.org/abs/1207.4109. The edge elimination ordering is then converted to a contractio plan which is then used to contract the tensor network.

(This method is still under development)

In [ ]:
circuit_network = create_basic_circuit(3)

QuickBB_contraction!(circuit_network)

### KaHyPar Contraction

The `KaHyPar_contraction!` function will contract a tensor network using the following top-down greedy method. First, the Karlsruhe Hypergraph Partitioning package (KaHyPar) is used to partition the nodes of the given tensor network into $k$ disjoint sets in a way that minimises the number of edges connecting the partitions (Other objective functions to be minimised, such as total edge dimension, to be added in future). The size of each parition will be at most $(1+\epsilon)$ times the average size of all partitions. 

Then a new tensor network is created, containing a single node for each partition and whose edges correspond to the edges connecting distinct partitions in the given tensor network. The netcon algorithm is then used to find the optimal contraction plan for this partition network. This contraction plan forms the final steps of the contraction plan for our TensorNetworkCircuit. To create the rest of the full contraction plan, the partition nodes in the contraction plan need to be replaced with the contraction plan for the associated partition/subnetwork. To do this, the described KaHyPar method is called recursively on the partitions. As a base case, if the number of nodes in a network is less than a given theshold $V$, then netcon is used to find the contraction plan for it.

(This method is still under development)

In [ ]:
circuit_network = create_basic_circuit(3)

k=2; ϵ=1; V=7

KaHyPar_contraction!(circuit_network, k, ϵ, V)

### Comparing Contraction Functions

The the quality of the contraction plans used/found by the different contraction methods may vary significantly if we change the network being contracted. We can compare the contraction plans by looking at the number of floating point operations it takes to contract the network according to a particular plan. Also, we could compare by looking at the maximum size of any intermediate tensors created by contracting the network. Both of these metrics are measured by the backend when a network is contracted. Below we illustrate a simple comparison of the contraction methods by repeatedly contracting the GHZ circuit for varying number of qubits.

In [ ]:
# A dictionary of the different contraction functions to compare.
contraction_functions = Dict(:full_wavefunction => full_wavefunction_contraction!,
                             :netcon => netcon_contraction!,
                             :bgreedy => bgreedy_contraction!)

# Maximum size of a GHZ circuit to contract.
max_qubits = 9

# Allocate memory for the metrics.
flops = Dict(label => Array{Real, 1}(undef, max_qubits) for label in keys(contraction_functions))
max_tensor_size = Dict(label => Array{Real, 1}(undef, max_qubits) for label in keys(contraction_functions))

# Use each contraction function to contract a range of circuits and record metrics.
for num_qubits = 1:max_qubits
    for (label, contraction_function) in contraction_functions
        
        network = create_basic_circuit(num_qubits)
        contraction_function(network)

        flops[label][num_qubits] = network.backend.metrics.flops
        max_tensor_size[label][num_qubits] = network.backend.metrics.max_tensor_size
            
    end
end

In [ ]:
using Plots

Below we plot the results of the above comparison

In [ ]:
# Create a plot comparing the floating point operations required by the different methods to contract the network
plt_flops = Plots.plot(title="flops", xlabel="number_of_qubits", ylabel="flops", legend=:topleft)
for function_label in keys(contraction_functions)
    plot!(plt_flops, 1:max_qubits, flops[function_label], label=String(function_label))
end
plt_flops


# Create a plot comparing the maximum size of a tensor required by the different methods to contract the network
plt_max_size = Plots.plot(title="max tensor size", xlabel="number_of_qubits", ylabel="max size", legend=:topleft)
for function_label in keys(contraction_functions)
    plot!(plt_max_size, 1:max_qubits, max_tensor_size[function_label], label=String(function_label))
end
plt_max_size


Plots.plot(plt_flops, plt_max_size, layout=(1,2))